# Vespa

>[Vespa](https://vespa.ai/) is a fully featured search engine and vector database. It supports vector search (ANN), lexical search, and search in structured data, all in the same query.

This notebook shows how to use `Vespa.ai` as a LangChain vector store.

In order to create a retriever, we use [pyvespa](https://pyvespa.readthedocs.io/en/latest/index.html) to
create a connection a `Vespa` service.

In [ ]:
#!pip install pyvespa

Using `pyvespa` package, you can either connect to a
[Vespa Cloud instance](https://pyvespa.readthedocs.io/en/latest/deploy-vespa-cloud.html)
or a local
[Docker instance](https://pyvespa.readthedocs.io/en/latest/deploy-docker.html).
Here, we will create a new Vespa application and deploy that using Docker.

First, we need to create an application package:

In [ ]:
from vespa.package import ApplicationPackage, Field, RankProfile

app_package = ApplicationPackage(name="testapp")
app_package.schema.add_fields(
    Field(name="text", type="string", indexing=["index", "summary"], index="enable-bm25"),
    Field(name="embedding", type="tensor<float>(x[384])",
          indexing=["attribute", "summary"],
          attribute=[f"distance-metric: angular"]),
)
app_package.schema.add_rank_profile(
    RankProfile(name="default",
                first_phase="closeness(field, embedding)",
                inputs=[("query(query_embedding)", "tensor<float>(x[384])")]
                )
)

This sets up a Vespa application with a schema for each document that contains
two fields: `text` for holding the document text and `embedding` for holding
the embedding vector. The `text` field is set up to use a BM25 index for
efficient text retrieval, and we'll see how to use this and hybrid search a
bit later.

The `embedding` field is set up with a vector of length 384 to hold the
embedding representation of the text. See
[Vespa's Tensor Guide](https://docs.vespa.ai/en/tensor-user-guide.html)
for more on tensors in Vespa.

Lastly, we add a [rank profile](https://docs.vespa.ai/en/ranking.html) to
instruct Vespa how to order documents. Here we set this up with a
[nearest neighbor search](https://docs.vespa.ai/en/nearest-neighbor-search.html).

Now we can deploy this application locally:

In [2]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
vespa_app = vespa_docker.deploy(application_package=app_package)

This deploys and creates a connection to a `Vespa` service. In case you
already have a Vespa application running for instance in the cloud,
please refer to the PyVespa application for how to connect.

Now, let's load some documents:

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

Here, we also set up local sentence embedder to transform the text to embedding
vectors. One could also use OpenAI embeddings, which we'll show a bit later.

To feed these to Vespa, we need to configure how the vector store should map to
fields in the Vespa application. Then we create the vector store directly from
this set of documents:

In [ ]:
vespa_config = dict(
    page_content_field="text",
    embedding_field="embedding",
    input_field="query_embedding"
)

from langchain.vectorstores import VespaStore

db = VespaStore.from_documents(docs, embedding_function, app=vespa_app, **vespa_config)

This creates a Vespa vector store and feeds that set of documents to Vespa.
The vector store takes care of calling the embedding function for each document
and inserts them into the database.

We can now query the vector store:

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

print(docs[0].page_content)

This will embed the query using the embedding function given above and query
Vespa. Note that this will use the `default` ranking function, which we set up
in the application package to be nearest neighbor ranking.

Please refer to the [pyvespa documentation](https://pyvespa.readthedocs.io/en/latest/getting-started-pyvespa.html#Query)
for more information.

Now you can return the results and continue using the results in LangChain.

